# 🦜⛓️ Langchain 101

In this notebook, we will explore the basic building blocks of the 🦜⛓️ Langchain framework, which is primarily used to create intelligent agents enabled by large language models. We will focus on practical examples to allow you, the participants, to try building such systems yourself.

This notebook is divided into the following main sections:
## Setup (2 min)

In this section, we will install all the required libraries needed throughout the workshop. Since we will use GPT (from OpenAI) as the main language model throughout the examples, you will be asked to provide the OpenAI API key.

## Motivation & Examples (5 min)

In this part, we will quickly provide some motivational examples and answer the question: "What is this good for?"

## Basic Components (15 min)

This section will focus on creating the first demonstration of an intelligent agent for personal home automation. We will explain the main components of the agent, such as tools, prompts, toolkits, as well as different types of pre-created and directly accessible agents. At the end of this section, multiple task exercises will be presented to improve our agent.

## Main Exercise (15 min)

In this section, you will be presented with an open-ended real-life problem that you can try to solve yourself.

## Additional Examples (10 min)

Here, we will showcase another real-life problem that is quite easily solvable with the use of LangChain.

___

## Links:
- [Langchain documentation](https://python.langchain.com/en/latest)
- [Colab basics](https://colab.research.google.com/?utm_source=scs-index)
- [Python language quick tutorial]( https://colab.research.google.com/github/data-psl/lectures2020/blob/master/notebooks/01_python_basics.ipynb)

## Setup (2 min)

In this section, we will install all the required libraries needed throughout the workshop.

In [ ]:
!pip install langchain openai google-search-results tiktoken faiss-cpu wikipedia > /dev/null

# # needed only in case we want to use custom llm models
# !pip install huggingface_hub > /dev/null
# !pip install transformers > /dev/null

Since we will use GPT (from OpenAI) as the main language model throughout the examples, you will be asked to provide an OpenAI API key.

You can get your API key [here](https://platform.openai.com/account/api-keys)

In [ ]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter openAI API key: ')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter openAI API key: ··········


This example also uses Serper as a way to search the internet for necesarry data.

You can create a free account [here](https://serper.dev) and get an API key with 1000 free requests [here](https://serper.dev/api-key)

In [ ]:
SERPER_API_KEY = getpass('Enter serpi API key: ')
os.environ['SERPER_API_KEY'] = SERPER_API_KEY

Enter serpi API key: ··········


Here we mount our shared google drive which contains the team directory notion file in CSV format

Don't forget to do it with your Sudolabs account :)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Motivation & Examples (5 min)

### Simple ReAct Agent

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0.25)

tools = load_tools(["google-serper", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("How old is Waren Buffet and what is his age multiplied by 2?")



> Entering new  chain...
 I need to find out how old Warren Buffet is and then do a calculation.
Action: google_serper
Action Input: Warren Buffet age
Observation: 92 years
Thought: I now need to multiply 92 by 2
Action: Calculator
Action Input: 92 * 2
Observation: Answer: 184
Thought: I now know the final answer
Final Answer: Warren Buffet is 92 years old and his age multiplied by 2 is 184.

> Finished chain.


'Warren Buffet is 92 years old and his age multiplied by 2 is 184.'

### CSV Agent

In [ ]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [ ]:
filepath = '/content/drive/Shareddrives/AI Workshops/Workshop 1 - LangChain/data/team_directory.csv'

llm = OpenAI(temperature=0)

agent = create_csv_agent(llm, filepath, verbose=True)

In [ ]:
agent.run("How many people are from Poprad?")



> Entering new  chain...
Thought: I need to find out how many people are from Poprad in the dataframe.
Action: python_repl_ast
Action Input: df[df['Location'] == 'Poprad'].shape[0]
Observation: 2
Thought: I now know the final answer
Final Answer: 2 people are from Poprad.

> Finished chain.


'2 people are from Poprad.'

In [ ]:
agent.run("How many different positions are in the engineering department?")



> Entering new  chain...
Thought: I need to find out how many different positions are in the engineering department
Action: python_repl_ast
Action Input: df[df['Department'] == 'Engineering']['Position'].nunique()
Observation: 9
Thought: I now know the final answer
Final Answer: 9

> Finished chain.


'9'

In [ ]:
agent.run("What is the distribution of positions?")



> Entering new  chain...
Thought: I need to find out how many of each position there are.
Action: python_repl_ast
Action Input: df['Position'].value_counts()
Observation: Engineer II                       13
Associate Engineer                12
Engineer III                       9
Engineer I                         8
Product Manager                    8
Product Designer                   5
Engineering Manager                4
Staff Engineer                     3
Social Media Marketing Manager     2
Senior engineer (deprecated)       2
Strategic Partnerships Manager     2
Head of Design                     1
CTO                                1
Social Media Manager               1
Community & Space Manager          1
Business Development Manager       1
Head of Product                    1
Associate Product Designer         1
Head of HR                         1
Head of marketing                  1
Senior Engineering Manager         1
CEO                                1
Chief of Stra

'There are 13 Engineer IIs, 12 Associate Engineers, 9 Engineer IIIs, 8 Engineer Is, 8 Product Managers, 5 Product Designers, 4 Engineering Managers, 3 Staff Engineers, 2 Social Media Marketing Managers, 2 Senior Engineers (deprecated), 2 Strategic Partnerships Managers, 1 Head of Design, 1 CTO, 1 Social Media Manager, 1 Community & Space Manager, 1 Business Development Manager, 1 Head of Product, 1 Associate Product Designer, 1 Head of HR, 1 Head of Marketing, 1 Senior Engineering Manager, 1 CEO, 1 Chief of Strategy, 1 CPO, 1 Product Analyst, 1 Business Operation Manager, 1 HR Officer, and 1 Recruiter/HR Specialist.'

In [ ]:
agent.run("What is the third most common position in the engineering department?")



> Entering new  chain...
Thought: I need to find the third most common position in the engineering department
Action: python_repl_ast
Action Input: df[df['Department'] == 'Engineering']['Position'].value_counts().index[2]
Observation: Engineer III
Thought: I now know the final answer
Final Answer: Engineer III

> Finished chain.


'Engineer III'

In [ ]:
agent.run("Who has the most senior position in the theexpert project?")



> Entering new  chain...
Thought: I need to find the person with the highest position in the theexpert project
Action: python_repl_ast
Action Input: df[df['Project'] == 'The Expert'].sort_values(by='Position', ascending=False).head(1)
Observation:     Unnamed: 0 Inactive           Name   Department        Position Location  \
36          36       No  Martin Dejmek  Engineering  Staff Engineer   Prague   

   LinkedIn                     Email Hobbies Start date GitHub Homepage  \
36      NaN  martindejmek@sudolabs.io     NaN        NaN    NaN      NaN   

       Project Twitter Gist Instagram StackOverflow Nickname  
36  The Expert     NaN  NaN       NaN           NaN      NaN  
Thought: I now know the final answer
Final Answer: Martin Dejmek

> Finished chain.


'Martin Dejmek'

### More agents [here](https://python.langchain.com/en/latest/modules/agents/toolkits.html)

## Basic Components (15 min)

This section will focus on creating the first demonstration of an intelligent agent for personal home automation. We will explain the main components of the agent, such as tools, prompts, toolkits, as well as different types of pre-created and directly accessible agents. At the end of this section, multiple task exercises will be presented to improve our agent.

The most important parts of the Langchain frameworks are:

- [**Models**](https://python.langchain.com/en/latest/modules/models/llms.html)
- [**Prompts**](https://python.langchain.com/en/latest/modules/prompts.html)
    - [Conceptual guide](https://docs.langchain.com/docs/components/prompts)
    - [Prompt templates](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)
- [**Chains**](https://python.langchain.com/en/latest/modules/chains.html)
    - [Conceptual Guide](https://docs.langchain.com/docs/components/chains)
    - [Intro to chains](https://python.langchain.com/en/latest/modules/chains/getting_started.html)
- [**Agents**](https://python.langchain.com/en/latest/modules/agents.html)
    - [Intro into agents](https://python.langchain.com/en/latest/modules/agents.html)
    - [Agent types](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html)
- [**Indexes**](https://python.langchain.com/en/latest/modules/indexes.html)
    - [Conceptual Guide](https://docs.langchain.com/docs/components/indexing)
    - [Intro to indexes](https://python.langchain.com/en/latest/modules/indexes/getting_started.html)
- [**Memory**](https://python.langchain.com/en/latest/modules/memory.html#)
    - [Conceptual Guide](https://docs.langchain.com/docs/components/memory)
    - [Intro into memory](https://python.langchain.com/en/latest/modules/memory/getting_started.html)



#### Prompts

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
impersonation_template = PromptTemplate(
    input_variables=["text"],
    template="Rewrite the folowing text like Yoda: {text}"
)

In [ ]:
explain_template = PromptTemplate.from_template("Explain this objective like I am a child in 20 words. Start the sentence with the name of the objective: {objective}")

#### Chains

In [ ]:
from langchain.chains import LLMChain

In [ ]:
explain_chain = LLMChain(llm=llm, prompt=explain_template)

In [ ]:
explain_chain.run({"objective": "Sky diving"})

'\n\nSky diving is jumping out of a plane and using a parachute to float down to the ground.'

In [ ]:
impersonation_chain = LLMChain(llm=llm, prompt=impersonation_template)

In [ ]:
impersonation_chain.run(text="Life is like a bar of chocolate.")

'\n\nA bar of chocolate, life is like.'

In [ ]:
from langchain.chains import SimpleSequentialChain

sequential_chain = SimpleSequentialChain(chains=[explain_chain, impersonation_chain], verbose=True)

In [ ]:
sequential_chain.run("mathematics")



> Entering new  chain...


Mathematics is learning how to solve problems using numbers and shapes.


Learning how to solve problems using numbers and shapes, mathematics is.

> Finished chain.


'\n\nLearning how to solve problems using numbers and shapes, mathematics is.'

### Exercise 1 - Basic components

Let's create chain that will retrieve current information from wikipedia and translate them to french.

[Wikipedia retriever docs](https://python.langchain.com/en/latest/modules/indexes/retrievers/examples/wikipedia.html)

In [ ]:
# ...live coding 🤞

### Exercise 1 - Solution

In [ ]:
from langchain.retrievers import WikipediaRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, SimpleSequentialChain, LLMChain
from langchain.prompts import PromptTemplate

retriever = WikipediaRetriever()

chain_with_retriever = RetrievalQA.from_chain_type(llm,retriever=retriever)

translate_template = PromptTemplate.from_template("Your task is to translate the following text to french: {text}")
translation_chain = LLMChain(llm=llm, prompt=translate_template)

sequential_chain = SimpleSequentialChain(chains=[chain_with_retriever, translation_chain], verbose=True)

sequential_chain.run("What is life?")



> Entering new  chain...
 "What Is Life" is a song by English rock musician George Harrison from his 1970 triple album All Things Must Pass. In many countries, it was issued as the second single from the album, in February 1971, becoming a top-ten hit in the United States, Canada and elsewhere, and topping singles charts in Australia and Switzerland. In the United Kingdom, "What Is Life" appeared as the B-side to "My Sweet Lord", which was the best-selling single there of 1971. Harrison's backing musicians on the song include Eric Clapton and the entire Delaney & Bonnie and Friends band, with whom he had toured during the final months of the Beatles. Harrison co-produced the recording with Phil Spector, whose Wall of Sound production also employed a prominent string arrangement by John Barham and multiple acoustic rhythm guitars, played by Harrison's fellow Apple Records signings Badfinger. An uptempo soul tune, "What Is Life" is one of several Harrison love songs that appear to be d

'\n\n"Qu\'est-ce que la vie" est une chanson du musicien de rock anglais George Harrison de son triple album de 1970 All Things Must Pass. Dans de nombreux pays, elle a été émise comme le deuxième single de l\'album, en février 1971, devenant un succès dans le top-ten aux États-Unis, au Canada et ailleurs, et atteignant les charts de singles en Australie et en Suisse. Au Royaume-Uni, "What Is Life" est apparue comme le B-side de "My Sweet Lord", qui était le single le plus vendu là-bas en 1971. Les musiciens de soutien de Harrison sur la chanson comprennent Eric Clapton et toute la bande Delaney & Bonnie and Friends, avec qui il avait fait une tournée pendant les derniers mois des Beatles. Harrison a co-produit l\'enregistrement avec Phil Spector, dont la production Wall of Sound comprenait également un arrangement de'

## Main exercise (15 min)

### Custom tools ReAct agents

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
import random

In [ ]:
class Home:
  rooms = ["living room", "bathroom", "bedroom", "kitchen", "library"]
  people = ["wolfgang", "antonius", "augustus", "peter"]

  room_people_mapping = {}
  people_room_mapping = {}
  room_temperature = {}
  room_windows = {}
  room_lights = {}

  def __init__(self):
    for person in self.people:
      random_room = random.choice(self.rooms)
      self.people_room_mapping[person] = random_room

    for room in self.rooms:
      self.room_people_mapping[room] = []
      self.room_temperature[room] = random.randint(20, 26)
      self.room_windows[room] = random.choice(['open', 'closed'])
      self.room_lights[room] = random.choice(['on', 'off'])

    for person, room in self.people_room_mapping.items():
      self.room_people_mapping[room].append(person)

  def _lowercase_input(func):
    def wrapper(self, room_name):
        if room_name is not None:
          room_name = room_name.lower()
          return func(self, room_name)
        else:
          func(self, "")
    return wrapper

  def _check_room_name(func):
    def wrapper(self, room_name):
        if room_name in self.rooms:
          return func(self, room_name)
        else:
          return f"There is no room named {room_name}"
    return wrapper

  def _describe_home(self):
    for room in self.rooms:
      print(f"""Room: {room}
            Lights: {self.room_lights[room]}
            Temperature: {self.room_temperature[room]}
            Windows: {self.room_windows[room]}
            People present: {self.room_people_mapping[room]}\n""")

  def get_all_rooms(self, dummy_placeholder=""):
    return f"Following rooms are present int the house: {self.rooms}"

  @_lowercase_input
  @_check_room_name
  def get_all_people_in_room(self, room_name):
    room_status = self.room_people_mapping.get(room_name)
    return f"In the {room_name} are following people: [{' ,'.join(room_status)}]"

  @_lowercase_input
  def find_person(self, person):
    person_location = self.people_room_mapping.get(person)
    if person_location == None:
      return f"There is no person named {person} in the house"
    return f"{person} is in the {person_location}"

  @_lowercase_input
  @_check_room_name
  def get_lights_status_in(self, room_name: str)-> str:
    light_status = self.room_lights.get(room_name)
    return f"The lights in the {room_name} are {light_status}"

  @_lowercase_input
  @_check_room_name
  def get_windows_status_in(self, room_name: str)-> str:
    windows_status = self.room_windows.get(room_name)
    return f"The windows in the {room_name} are {windows_status}"

  @_lowercase_input
  @_check_room_name
  def get_temperature_in(self, room_name: str)-> str:
    temperature = self.room_temperature.get(room_name)
    return f"The tempereature in the {room_name} {temperature} °C"

  @_lowercase_input
  @_check_room_name
  def turn_on_lights_in(self, room_name):
    self.room_lights[room_name] = 'on'
    return f"Turning on lights in {room_name}"

  @_lowercase_input
  @_check_room_name
  def turn_off_lights_in(self, room_name):
    self.room_lights[room_name] = 'off'
    return f"Turning off lights in {room_name}"

  @_lowercase_input
  @_check_room_name
  def open_windows_in(self, room_name):
    self.room_windows[room_name] = 'open'
    return f"Windows in the {room_name} are now opened"

  @_lowercase_input
  @_check_room_name
  def close_windows_in(self, room_name):
    self.room_windows[room_name] = 'closed'
    return f"Windows in the {room_name} are now closed"

In [ ]:
my_home = Home()

In [ ]:
my_home._describe_home()

Room: living room
            Lights: on
            Temperature: 20
            Windows: closed
            People present: ['antonius', 'augustus']

Room: bathroom
            Lights: off
            Temperature: 20
            Windows: closed
            People present: ['wolfgang']

Room: bedroom
            Lights: off
            Temperature: 24
            Windows: closed
            People present: ['peter']

Room: kitchen
            Lights: off
            Temperature: 20
            Windows: closed
            People present: []

Room: library
            Lights: on
            Temperature: 25
            Windows: closed
            People present: []



In [ ]:
print(my_home.get_all_rooms())
print(my_home.get_all_people_in_room("living room"))
print(my_home.find_person("wolfgang"))
print(my_home.get_lights_status_in("living room"))
print(my_home.get_temperature_in("living room"))
print(my_home.get_lights_status_in("living room"))

Following rooms are present int the house: ['living room', 'bathroom', 'bedroom', 'kitchen', 'library']
In the living room are following people: [antonius ,augustus]
wolfgang is in the bathroom
The lights in the living room are on
The tempereature in the living room 20 °C
The lights in the living room are on


In [ ]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

tools = [
    Tool.from_function(
        func=my_home.get_all_people_in_room,
        name = "Get people in room",
        description="useful for when you need to find out what people are located in specific room as well as finding out whether specific room is empty or not. takes name of the room as parameter"
    ),
    Tool.from_function(
        func=my_home.get_all_rooms,
        name = "List all rooms",
        description="useful for when you need to find out what rooms are available in home. The function returns a list of available rooms"
    ),
    Tool.from_function(
        func=my_home.find_person,
        name = "Find person",
        description="useful for when you need to find where is certain person in the home. The input for this function is the individual room. (ex: living room, bathroom)"
    ),
    Tool.from_function(
        func=my_home.turn_on_lights_in,
        name="Turn on lights in specific room",
        description="useful for turning on the light in specific room. The input for this function is the individual room. (ex: living room, bathroom)"
    ),
    Tool.from_function(
        func=my_home.turn_off_lights_in,
        name="Turn off lights in specific room",
        description="useful for turning off the light in specific room. The input for this function is the individual room. (ex: living room, bathroom)"
    ),
    Tool.from_function(
        func=my_home.get_lights_status_in,
        name="Get status[on/off] lights in specific room",
        description="useful for finding out if the lights are on / off in specific room. The input for this function is the individual room. (ex: living room, bathroom)"
    )
]

In [ ]:
llm = OpenAI(temperature=0.25)

home_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
home_agent.run("Where is Wolfgang?")



> Entering new  chain...
 I need to find out where Wolfgang is located.
Action: Find person
Action Input: Wolfgang
Observation: wolfgang is in the kitchen
Thought: I now know the final answer
Final Answer: Wolfgang is in the kitchen.

> Finished chain.


'Wolfgang is in the kitchen.'

In [ ]:
home_agent.run("trn of light in he kitchn") # intentional typos



> Entering new  chain...
 I need to turn off the lights in the kitchen
Action: Turn off lights in specific room
Action Input: kitchen
Observation: Turning off lights in kitchen
Thought: I now know the final answer
Final Answer: Lights in the kitchen have been turned off.

> Finished chain.


'Lights in the kitchen have been turned off.'

In [ ]:
home_agent.run("If there is someone in the living room, you should turn on the lights, if not, do not turn on the lights.")



> Entering new  chain...
 I need to find out if someone is in the living room.
Action: Get people in room
Action Input: Living room
Observation: In the living room are following people: []
Thought: There is no one in the living room.
Action: Turn off lights in specific room
Action Input: Living room
Observation: Turning off lights in living room
Thought: I now know the final answer.
Final Answer: Do not turn on the lights in the living room.

> Finished chain.


'Do not turn on the lights in the living room.'

In [ ]:
home_agent.run("Is there somone in the house?")



> Entering new  chain...
 I need to find out who is in the house.
Action: Get people in room
Action Input: All rooms
Observation: There is no room named all rooms
Thought: I need to find out which rooms are available
Action: List all rooms
Action Input: N/A
Observation: Following rooms are present int the house: ['living room', 'bathroom', 'bedroom', 'kitchen', 'library']
Thought: I need to check if anyone is in any of these rooms
Action: Find person
Action Input: living room, bathroom, bedroom, kitchen, library
Observation: There is no person named living room, bathroom, bedroom, kitchen, library in the house
Thought: I need to check if anyone is in any of these rooms
Action: Get people in room
Action Input: living room, bathroom, bedroom, kitchen, library
Observation: There is no room named living room, bathroom, bedroom, kitchen, library
Thought: I need to check each room individually
Action: Get people in room
Action Input: living room
Observation: In the living room are followin

'Yes, there are people in the house.'

In [ ]:
home_agent.run("Find all people in the house")



> Entering new  chain...
 I need to find out which rooms are available and then find out who is in each room
Action: List all rooms
Action Input: N/A
Observation: Following rooms are present int the house: ['living room', 'bathroom', 'bedroom', 'kitchen', 'library']
Thought: I need to find out who is in each room
Action: Get people in room
Action Input: living room
Observation: In the living room are following people: []
Thought: I need to repeat this for all the rooms
Action: Get people in room
Action Input: bathroom
Observation: In the bathroom are following people: []
Thought: I need to repeat this for all the rooms
Action: Get people in room
Action Input: bedroom
Observation: In the bedroom are following people: [antonius ,peter]
Thought: I need to repeat this for all the rooms
Action: Get people in room
Action Input: kitchen
Observation: In the kitchen are following people: [wolfgang ,augustus]
Thought: I need to repeat this for all the rooms
Action: Get people in room
Action In

'The people in the house are antonius, peter, wolfgang, and augustus.'

### Exercise 2 - Agents and their Tools

Add support for window and temperature management for queries like:
- "If the temperature in the living room is less than 21, close the windows"

### Exercise 2 - Solution

### Bonus: Custom agent with custom tools

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
import re

In [ ]:
class Home:
  rooms = ["living room", "bathroom", "bedroom", "kitchen", "library"]
  people = ["wolfgang", "antonius", "augustus", "peter"]

  room_people_mapping = {}
  people_room_mapping = {}
  room_temperature = {}
  room_windows = {}
  room_lights = {}

  def __init__(self):
    for person in self.people:
      random_room = random.choice(self.rooms)
      self.people_room_mapping[person] = random_room

    for room in self.rooms:
      self.room_people_mapping[room] = []
      self.room_temperature[room] = random.randint(20, 26)
      self.room_windows[room] = random.choice(['open', 'closed'])
      self.room_lights[room] = random.choice(['on', 'off'])

    for person, room in self.people_room_mapping.items():
      self.room_people_mapping[room].append(person)

  def _lowercase_input(func):
    def wrapper(self, room_name):
        if room_name is not None:
          room_name = room_name.lower()
          return func(self, room_name)
        else:
          func(self, "")
    return wrapper

  def _check_room_name(func):
    def wrapper(self, room_name):
        if room_name in self.rooms:
          return func(self, room_name)
        else:
          return f"There is no room named {room_name}"
    return wrapper

  def _describe_home(self):
    for room in self.rooms:
      print(f"""Room: {room}
            Lights: {self.room_lights[room]}
            Temperature: {self.room_temperature[room]}
            Windows: {self.room_windows[room]}
            People present: {self.room_people_mapping[room]}\n""")

  def get_all_rooms(self, dummy_placeholder=""):
    return f"Following rooms are present int the house: {self.rooms}"

  @_lowercase_input
  @_check_room_name
  def get_all_people_in_room(self, room_name):
    room_status = self.room_people_mapping.get(room_name)
    return f"In the {room_name} are following people: [{' ,'.join(room_status)}]"

  @_lowercase_input
  def find_person(self, person):
    person_location = self.people_room_mapping.get(person)
    if person_location == None:
      return f"There is no person named {person} in the house"
    return f"{person} is in the {person_location}"

  @_lowercase_input
  @_check_room_name
  def get_lights_status_in(self, room_name: str)-> str:
    light_status = self.room_lights.get(room_name)
    return f"The lights in the {room_name} are {light_status}"

  @_lowercase_input
  @_check_room_name
  def get_windows_status_in(self, room_name: str)-> str:
    windows_status = self.room_windows.get(room_name)
    return f"The windows in the {room_name} are {windows_status}"

  @_lowercase_input
  @_check_room_name
  def get_temperature_in(self, room_name: str)-> str:
    temperature = self.room_temperature.get(room_name)
    return f"The tempereature in the {room_name} {temperature} °C"

  @_lowercase_input
  @_check_room_name
  def turn_on_lights_in(self, room_name):
    self.room_lights[room_name] = 'on'
    return f"Turning on lights in {room_name}"

  @_lowercase_input
  @_check_room_name
  def turn_off_lights_in(self, room_name):
    self.room_lights[room_name] = 'off'
    return f"Turning off lights in {room_name}"

  @_lowercase_input
  @_check_room_name
  def open_windows_in(self, room_name):
    self.room_windows[room_name] = 'open'
    return f"Windows in the {room_name} are now opened"

  @_lowercase_input
  @_check_room_name
  def close_windows_in(self, room_name):
    self.room_windows[room_name] = 'closed'
    return f"Windows in the {room_name} are now closed"

In [ ]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

tools = [
    Tool.from_function(
        func=my_home.get_all_people_in_room,
        name = "Search people",
        description="useful for when you need to find out what people are located in specific romm as well as finding out wheter specific room is empty or not. takes name of the room as a parameter"
    ),
    Tool.from_function(
        func=my_home.get_all_rooms,
        name = "List rooms",
        description="useful for when you need to find out what rooms are available in home. The input for this function is the individual room"
    ),
    Tool.from_function(
        func=my_home.find_person,
        name = "Find occupant",
        description="useful for when you need to find in which room a person is"
    ),
    Tool.from_function(
        func=my_home.turn_on_lights_in,
        name="Turn on lights in specific room",
        description="useful for turning on the light in specific room"
    ),
    Tool.from_function(
        func=my_home.turn_off_lights_in,
        name="Turn off lights in specific room",
        description="useful for turning off the light in specific room"
    )
]

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
# Set up the base template
template = """You are the home automaition agent that helps users with gathering information about
their homes as wella s heling them with automation. You have access to the following tools:

{tools}

This is the history of your conversation with the human:

{chat_history}

Use the following format:

Comand: the input command you must answer / make action
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final: the final answer / action to the original input command

Begin!

Question: {input}
{agent_scratchpad}"""

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "chat_history"]
)

In [ ]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)

In [ ]:
llm = OpenAI(temperature=0.25)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=readonlymemory
)

In [ ]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    memory=memory,
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=5
)

In [ ]:
agent_executor.run("Where is Augustus?")



> Entering new  chain...

Thought: I need to find out where Augustus is located.
Action: Find occupant
Action Input: Augustus

Observation:augustus is in the bathroom
 I now know the final answer
Final: Augustus is in the bathroom

> Finished chain.


'I now know the final answer\nFinal: Augustus is in the bathroom'

In [ ]:
print(agent_executor.memory.buffer)

Human: Where is Augustus?
AI: I now know the final answer
Final: Augustus is in the bathroom


In [ ]:
agent_executor.run("Turn off the lights in that room where is augustus?")

Thought: I need to find out which room Augustus is in and then turn off the lights in that room
Action: Find occupant
Action Input: Augustus

Observation:kitchen
 I now need to turn off the light in the kitchen 
Action: Turn off lights in specific room
Action Input: kitchen

Observation:Turning off lights in kitchen
 I now know the lights are off in the kitchen
Final: Lights are off in the kitchen.

> Finished chain.


'I now know the lights are off in the kitchen\nFinal: Lights are off in the kitchen.'

## Additional exercise (10 min)

### Exercise 3 - Document Q&A

Let's create a system that that answers questions from some very long texts(books, movie scripts, software documentations, meeting transcriptions, etc.)

Hint - the solution should contain these parts:
- [Loader](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) - loads your files into Documents
- [Splitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) - splits your long text into chunks
- [Embedding model](https://python.langchain.com/en/latest/modules/models/text_embedding.html) - converts your short texts into embeddings
- [Vectorstore](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) - database for your embeddings
- [RetrievalQA chain](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) - the main 🦜⛓️ part

### Exercise 3 - Solution

Sources:
- [IMSdb loader](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/imsdb.html)
- [Recursive Text Splitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)
- [FAISS Vectorstore](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html)
- [Question asnwering chain](https://python.langchain.com/en/latest/modules/chains/index_examples/question_answering.html)
- [Retrieval and QA chain](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html)


#### Loader

Fetch the PulpFiction movie script from [IMSdb](https://imsdb.com)

In [ ]:
from langchain.document_loaders import IMSDbLoader

loader = IMSDbLoader("https://imsdb.com/scripts/Pulp-Fiction.html")

documents = loader.load()

print(len(documents))

1


You can see here that the whole script contains 304 709 characters.

In [ ]:
print(type(documents[0]))
print(len(documents[0].page_content))

<class 'langchain.schema.Document'>
304709


The script contains a lot of empty spaces which we will remove here.

However the remaining lenght of the script is still very long, 169 541 characters long.

In [ ]:
from langchain.docstore.document import Document

cleanText = documents[0].page_content.replace("  ", "")
document = Document(page_content=cleanText, metadata=documents[0].metadata)

print(len(document.page_content))

169541


#### Splitter

We simply split the whole script into smaller chunks of 1000 characters that overlap by 200 characters.

This is a very lazy and naive way of splitting long documents. We might accidentaly lose important information that spans across 1000 characters which will make the question asnwering more difficult.

More sophisticated approach would be to split the movie script into some logical sequences like scenes, shots, dialogues.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

docs = text_splitter.split_documents([document])

len(docs)

213

#### Embedding model & Vectorstore

This sections embeds the documents with the default OpenAI encoder model and saves their embeddings into a [FAISS](https://github.com/facebookresearch/faiss) db (langchain calls it a vectorstore).

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

This is just a double check that the embeddings are correct and the database works.

If we take any line from the script, this section should find the document that contains the line.

In [ ]:
query = "Uncomfortable silences. Why do we feel it's necessary to yak about bullshit in order to be comfortable?"
results = db.similarity_search_with_score(query, k=1)

for doc, score in results:
  print(score)
  print(doc.page_content)

0.34167325
kooties.

 Vincent smiles.

 VINCENT
 Yeah, but maybe I do.

 MIA
 Kooties I can handle.

 He takes a sip.

 VINCENT
 Goddamn! That's a pretty fuckin' 
 good milk shake.

 MIA
 Told ya.

 VINCENT
 I don't know if it's worth five 
 dollars, but it's pretty fuckin' 
 good.

 He slides the shake back.

 Then the first of an uncomfortable silence happens.

 MIA
 Don't you hate that?

 VINCENT
 What?

 MIA
 Uncomfortable silences. Why do we 
 feel it's necessary to yak about 
 bullshit in order to be comfortable?

 VINCENT
 I don't know.

 MIA
 That's when you know you found 
 somebody special. When you can just 
 shit the fuck up for a minute, and 
 comfortably share silence.

 VINCENT
 I don't think we're there yet. But 
 don't feel bad, we just met each 
 other.

 MIA
 Well I'll tell you what, I'll go to 
 the bathroom and powder my nose, 
 while you sit here and think of 
 something to say.

 VINCENT
 I'll do that.


#### RetrievalQA chain

This is the part where the answering happens.

The RetrievalQA chain firstly finds the 4 most relevant documents to our query and sends it to the LLM as a context. The LLM is then asked to find an answer in this context.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever, return_source_documents=False)

#### Example questions

In [ ]:
query = "How is a quarter pounder with cheese called in france?"
qa.run(query)

' A quarter pounder with cheese is called a Royale with Cheese in France.'

In [ ]:
query = "What do they put on fries in holland?"
qa.run(query)

' They put mayonnaise on fries in Holland.'

In [ ]:
query = "What is the passage from bible that Jules has memorized?"
qa.run(query)

' Ezekiel 25:17. "The path of the righteous man is beset on all sides by the inequities of the selfish and the tyranny of evil men. Blessed is he who, in the name of charity and good will, shepherds the weak through the valley of darkness, for he is truly his brother\'s keeper and the finder of lost children. And I will strike down upon thee with great vengeance and furious anger those who attempt to poison and destroy my brothers. And you will know my name is the Lord when I lay my vengeance upon you."'

In [ ]:
query = "Where do Vincent and Mia have dinner?"
qa.run(query)

" Vincent and Mia have dinner at Jackrabbit Slim's."

In [ ]:
query = "What is the name of the goon that Vincent accidentally shoots in the face?"
qa.run(query)

' Marvin'